In [1]:
import networkx as nx
import numpy as np
import pandas as pd 
import operator
import matplotlib.pyplot as plt

In [2]:
nx_graphs_learning = []
nx_graphs_predict = []
agg_graph = nx.MultiGraph()
venue_df_list = []
for i in range(20):
	# with open(f'..\ATX_Networks\sim_day_{i+1}.gexf') as graph_file:
	nx_graphs_learning.append(nx.read_gexf(f'..\ATX_Networks\sim_day_{i+1}.gexf'))
	venue_dict = nx.get_edge_attributes(nx_graphs_learning[i], "venueid")
	venue_arr = list(venue_dict.items())
	rows = []
	for row in venue_arr:
		rows.append([row[0][0],row[0][1],row[1]])
	venue_df_list.append(pd.DataFrame(rows))


In [3]:
venue_df_list_predict = []
for i in range(19,31):
    nx_graphs_predict.append(nx.read_gexf(f'..\ATX_Networks\sim_day_{i+1}.gexf'))
    venue_dict = nx.get_edge_attributes(nx_graphs_predict[i-19], "venueid")
    venue_arr = list(venue_dict.items())
    rows = []
    for row in venue_arr:
        rows.append([row[0][0],row[0][1],row[1]])
    venue_df_list_predict.append(pd.DataFrame(rows))

In [15]:
concat_venue_df_learn = pd.concat(venue_df_list)
concat_venue_df_learn.columns = ['source','target','venueid']
learn_edges = concat_venue_df_learn.iloc[:,:2]
concat_venue_df_learn.head()

,source,target,venueid
0,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,440da2cbf964a52091301fe3
1,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,cf4e564f4c8e3aecc9dacafce90cada3eabaff58464e74...,440da2cbf964a52091301fe3
2,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,594a39e074fda9219102eff386f3da1944494aaa18c036...,440da2cbf964a52091301fe3
3,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,a1470e73f1bc4f7c757c49138afd84bd1ad09e9277526a...,440da2cbf964a52091301fe3
4,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,6f8848838f9afe5a7b38c23ad5018179f69e518ee721ba...,440da2cbf964a52091301fe3


In [14]:
concat_venue_df_pred = pd.concat(venue_df_list_predict)
concat_venue_df_pred.columns = ['source','target','venueid']
pred_edges = concat_venue_df_pred.iloc[:,:2]
pred_edges.head()

,source,target
0,2948c798ef6107e35f8624de3150851c739055bd3db6c3...,47e7f42109db6e5036850a024106f67d79480befd0788b...
1,1466d9c36f2eb213835513d860cdd692cf2d11f5f5ce98...,6049f76add3b013b811bd7bdc2f9a14b6e651307d59a39...
2,a7236c8817c615d21bcc3d2c1c252a6f1e5a2a262b99b9...,6581e0679fc7675654aa43b2329be2b4de689503806e25...
3,a7236c8817c615d21bcc3d2c1c252a6f1e5a2a262b99b9...,7a61d0b7162468df8a51a5d7608070310ef737a9c5ce63...
4,83ea847f734e5c68c80a12ee564421610f7a1ac44e965a...,76a48312371a24467062b2e1df331905003cb23b6b5fad...


In [16]:
print(concat_venue_df_learn.shape)
print(concat_venue_df_pred.shape)

(143543, 3)
(93410, 3)


In [17]:
ds_learn = set(map(tuple, learn_edges.values))
ds_pred = set(map(tuple, pred_edges.values))
ds_to_predict = pd.DataFrame(list(ds_pred.difference(ds_learn)))
ds_to_predict.columns=['source', 'target']

In [18]:
ds_to_predict.head()

,source,target
0,104fc132c04993e6955884fb2ab97b0844e50c1c81cd63...,04db9d32ea5e47b9488526d0502ca13a8dcf95f3aec2b4...
1,f73cbff4ce8020326fd7cdfc0b828d7b4a10c2bd23332e...,9449224cea99c7e5d840c09574aa2d5398d8c1340bc4a4...
2,d475793dd47954d158bbaa35817d94fe8d3951143d7075...,58b01c4a7b3896c95f59e9259d625eb68c89e863a08fcc...
3,d8e4658adb2cd20d208eb6f089e4c482bf7ec7fa084ea9...,7cada410228c4410d1467fde511a5e341ce4153a69c402...
4,701d30f27393ee745ac72a2d6143fac7b4ee0f4bf923c5...,c87eca7fbe10670a92cfa47665a998b081c80a032e57b1...


In [19]:
print(ds_to_predict.shape)
print(len(ds_learn), len(ds_pred))

(81796, 2)
137879 90050


There are 81796 unique colocations in the last 11 days that never happened in the first 20 days. 